# Command and Control Application Suite E2E Demonstration

© Crown-owned copyright 2024, Defence Science and Technology Laboratory UK

This notebook demonstrates the current implementation of the command and control (C2) server and beacon applications in primAITE.

In [ ]:
# Imports
from primaite.config.load import data_manipulation_config_path
from primaite.session.environment import PrimaiteGymEnv
from primaite.game.agent.interface import AgentHistoryItem
import yaml
from pprint import pprint
from primaite.simulator.network.container import Network
from primaite.game.game import PrimaiteGame
from primaite.simulator.system.applications.red_applications.c2.c2_beacon import C2Beacon
from primaite.simulator.system.applications.red_applications.c2.c2_server import C2Server
from primaite.simulator.system.applications.red_applications.c2.abstract_c2 import C2Command, C2Payload
from primaite.simulator.system.applications.red_applications.ransomware_script import RansomwareScript
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server

## **Notebook Setup** | **Network Configuration:**

This notebook uses the same network setup as UC2. Please refer to the main [UC2-E2E-Demo notebook for further reference](./Data-Manipulation-E2E-Demonstration.ipynb).

However, this notebook will replaces with the red agent used in UC2 with a custom proxy red agent built for this notebook.

In [ ]:
custom_c2_agent = """
  - ref: CustomC2Agent
    team: RED
    type: ProxyAgent
    observation_space: null
    action_space:
      action_list:
        - type: DONOTHING
        - type: NODE_APPLICATION_INSTALL
        - type: NODE_APPLICATION_EXECUTE
        - type: CONFIGURE_C2_BEACON
        - type: C2_SERVER_RANSOMWARE_LAUNCH
        - type: C2_SERVER_RANSOMWARE_CONFIGURE
        - type: C2_SERVER_TERMINAL_COMMAND
      options:
        nodes:
          - node_name: client_1
            applications: 
              - application_name: C2Beacon
          - node_name: domain_controller
            applications: 
              - application_name: C2Server
        max_folders_per_node: 1
        max_files_per_folder: 1
        max_services_per_node: 2
        max_nics_per_node: 8
        max_acl_rules: 10
        ip_list:
          - 192.168.1.10
          - 192.168.1.14
      action_map:
        0:
          action: DONOTHING
          options: {}
        1:
          action: NODE_APPLICATION_INSTALL
          options:
            node_id: 0
            application_name: C2Beacon
        2:
          action: CONFIGURE_C2_BEACON
          options:
            node_id: 0
            config:
              c2_server_ip_address: 192.168.1.10
              keep_alive_frequency:
              masquerade_protocol:
              masquerade_port:
        3:
          action: NODE_APPLICATION_EXECUTE
          options:
            node_id: 0
            application_id: 0  
        4:
          action: NODE_APPLICATION_INSTALL
          options:
            node_id: 0
            application_name: RansomwareScript  
        5:
          action: C2_SERVER_RANSOMWARE_CONFIGURE
          options:
            node_id: 1
            config:
              server_ip_address: 192.168.1.14
              payload: ENCRYPT
        6:
          action: C2_SERVER_RANSOMWARE_LAUNCH
          options:
            node_id: 1
        7:
          action: C2_SERVER_TERMINAL_COMMAND
          options:
            node_id: 1
            application_id: 0   

    reward_function:
      reward_components:
        - type: DUMMY
"""
c2_agent_yaml = yaml.safe_load(custom_c2_agent)

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = c2_agent_yaml
    

env = PrimaiteGymEnv(env_config=cfg)

## **Notebook Setup** | Network Prerequisites

Before the Red Agent is able to perform any C2 specific actions, the C2 Server needs to be installed and run before the episode begins.

This is because higher fidelity environments (and the real-world) a C2 server would not be accessible by private network blue agent and the C2 Server would already be in place before the an adversary (Red Agent) before the narrative of the use case.

The cells below installs and runs the C2 Server on the domain controller server directly via the simulation API.

In [ ]:
domain_controller: Server = env.game.simulation.network.get_node_by_hostname("domain_controller")
domain_controller.software_manager.install(C2Server)
c2_server: C2Server = domain_controller.software_manager.software["C2Server"]
c2_server.run()
domain_controller.software_manager.show()

## **Command and Control** | C2 Beacon Actions

Before the Red Agent is able to perform any C2 Server commands, it must first establish connection with a C2 beacon.

This can be done by installing, configuring and then executing a C2 Beacon. 

In [ ]:
c2_red_agent = env.game.agents["CustomC2Agent"]
client_1: Computer = env.game.simulation.network.get_node_by_hostname("client_1")


### **Command and Control** | C2 Beacon Actions | Installation

In [ ]:
env.step(1)
client_1.software_manager.show()
c2_beacon: C2Beacon = client_1.software_manager.software["C2Beacon"]

### **Command and Control** | C2 Beacon Actions | Configuration

In [ ]:
env.step(2)
c2_beacon.show()

### **Command and Control** | C2 Beacon Actions | Establishing Connection

In [ ]:
env.step(3)

In [ ]:
c2_beacon.show()
c2_server.show()

## **Command and Control** | C2 Server Actions

### **Command and Control** | C2 Server Actions | Configuring Ransomware

In [ ]:
env.step(4)

In [ ]:
env.step(5)

In [ ]:
ransomware_script: RansomwareScript = client_1.software_manager.software["RansomwareScript"]
client_1.software_manager.show()
ransomware_script.show()

### **Command and Control** | C2 Server Actions | Launching Ransomware

In [ ]:
env.step(6)

In [ ]:
database_server: Server = env.game.simulation.network.get_node_by_hostname("database_server")
database_server.software_manager.file_system.show(full=True)

### **Command and Control** | C2 Server Actions | Executing Terminal Commands

In [ ]:
# TODO: Post Terminal.
#env.step(7)